In [5]:

import torch
from speechbrain.pretrained import EncoderDecoderASR


# configure data path
train_metadata_filepath = "D:\LibriMix\data\Libri1Mix\wav8k\min\metadata\mixture_libri2mix_train-100_mix_single.csv"
dev_metadata_filepath = "D:\LibriMix\data\Libri1Mix\wav8k\min\metadata\mixture_libri2mix_dev_mix_single.csv"
test_metadata_filepath = "D:\LibriMix\data\Libri1Mix\wav8k\min\metadata\mixture_libri2mix_test_mix_single.csv"

# load pretrained model
run_opts = {"device": "cuda:0" if torch.cuda.is_available() else "cpu"}
asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech",
                                           savedir="./pretrained_ASR", run_opts=run_opts)

In [6]:
import sys
import pandas as pd
from wer import wer


def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()


test_metadata = pd.read_csv(test_metadata_filepath)
errors = []
for row in test_metadata.iterrows():
    progress(row[0], test_metadata.shape[0])
    test_sample = row[1]
    hypothesis = asr_model.transcribe_file(test_sample['mixture_path'])
    errors.append(wer(test_sample['transcript'].split(), hypothesis.split()))
print()
print(sum(errors) / len(errors))

[============================================================] 100.0% ...
43.80766651047766
